# Generate RGB images from FITS files and prepare images for pix2pix algorithm

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits

import sunpy
import sunpy.map
import matplotlib.colors as colors
from astropy import *
import astropy.units as u
import pathlib
import json, urllib, numpy as np, matplotlib.pylab as plt, matplotlib.ticker as mtick, requests
import sunpy.map
import drms
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
from sunpy.cm import color_tables as ct
import sunpy.coordinates
from datetime import datetime as dt_obj
import matplotlib.dates as mdates
import matplotlib.colors as mcol
import matplotlib.patches as ptc
from matplotlib.dates import *
import math

import drms
c = drms.Client()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
aia_dir = '2012/2012_fits_AIA0304'
hmi_dir = '2012/2012_fits_HMI'
aia_files = sorted(os.listdir(aia_dir))
hmi_files = sorted(os.listdir(hmi_dir))

In [ ]:
# generate AIA images
num = 0
for file in aia_files[:]:
    try:
        aia_map = sunpy.map.Map(os.path.join(aia_dir, file)).resample((1024, 1024)*u.pix)
        figure = plt.figure(frameon=False)
        axes = plt.Axes(figure, [0., 0., 1., 1.])
        axes.set_axis_off()
        figure.add_axes(axes)
        im = aia_map.plot(axes=axes, annotate=False)
        im.write_png(f'2012/2012images/AIA0304/{num}.png')
        num = num + 1
    except:
        num + num + 1

In [ ]:
# generate HMI images

import matplotlib.pyplot as plt
from reproject import reproject_interp

import astropy.units as u

import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a
from PIL import Image
import cv2

In [ ]:
# create all HMI (align, centering, apply mask and create png)
num = 0
for (aia, hmi) in  zip(aia_files[:], hmi_files[:]):
    try:
        hmi_map = sunpy.map.Map(os.path.join(hmi_dir, hmi)).resample((1024, 1024)*u.pix)
        aia_map = sunpy.map.Map(os.path.join(aia_dir, aia)).resample((1024, 1024)*u.pix)

        output, footprint = reproject_interp(hmi_map, aia_map.wcs, aia_map.data.shape)

        out_hmi = sunpy.map.Map(output, aia_map.wcs)

        figure = plt.figure(frameon=False)
        axes = plt.Axes(figure, [0., 0., 1., 1.])

        axes.set_axis_off()
        figure.add_axes(axes)

        img = out_hmi.plot(axes=axes, annotate=False)

        img.write_png(f'2012/2012images/HMI/{num}.png')

        im = cv2.imread(f'2012/2012images/HMI/{num}.png')
        height,width,depth = im.shape
        circle_img = np.zeros((height,width), np.uint8)
        cv2.circle(circle_img,(int(width/2),int(height/2)),1024,1,thickness=-1)
        masked_data = cv2.bitwise_and(im, im, mask=circle_img)

        im = Image.fromarray(masked_data)
        im.save(f'2012/2012images/HMI/{num}.png')
        num += 1
    except:
        num += 1

In [ ]:
# convert RGBA to RGB
num = 0
for i, img in enumerate(natsorted(os.listdir('2012/2012images/AIA0304/'))):
    img = Image.open(f'2012/2012images/AIA0304/{img}')
    rgb_image = img.convert('RGB')
    rgb_image.save(f'2012/2012images/AIA0304/{num}.png')
    num += 1

In [ ]:
# pix2pix image pairs preparation

import os
import numpy as np
from PIL import Image
import cv2
from natsort import natsorted

# define paths for translation from domain A (images in folderA) -> domain B (images in folderB)
folderA = '2012/2012images/HMI'
folderB = '2012/2012images/AIA0304'
dest_path = '2012/2012images/output'

splits = os.listdir(folderA)

for sp in splits:
    img_fold_A = os.path.join(folderA, sp)
    img_fold_B = os.path.join(folderB, sp)
    img_list = natsorted(os.listdir(img_fold_A))
    num_imgs = len(img_list)
    img_fold_AB = os.path.join(dest_path, sp)
    if not os.path.isdir(img_fold_AB):
        os.makedirs(img_fold_AB)
    print('split = %s, number of images = %d' % (sp, num_imgs))
    number = 1600
    for n in range(num_imgs):
        name_A = img_list[n]
        path_A = os.path.join(img_fold_A, name_A)
        name_B = name_A
        path_B = os.path.join(img_fold_B, name_B)
        if os.path.isfile(path_A) and os.path.isfile(path_B):
            name_AB = name_A
            path_AB = os.path.join(img_fold_AB, name_AB)
            im_A1 = Image.open(path_A)
            im_A = np.array(im_A1)
            im_B1 = Image.open(path_B)
            im_B = np.array(im_B1)
            im_AB = np.concatenate([im_A, im_B], 1)
            im_AB = Image.fromarray(im_AB)
            im_AB.save(f'2012/2012images/output/train/{number}.png')
            number += 1